SQL Server Databases Inventory

In [4]:
# Read data from SQL Server
import pyodbc
import sqlalchemy as sa
import numpy as np
import pandas as pd
from sqlalchemy.engine import URL       # To support pandas
from sqlalchemy import create_engine    # To support pandas
import openpyxl
from openpyxl.styles import PatternFill
import xlsxwriter
#import seaborn as sns
#import matplotlib.pyplot as plt
#from IPython.display import display

# Enter no. of days for report
days=7
#days = input("Enter no. of days to search: ")
#print(days)

sheet_no=1
# Initialize the list size
appended_data = [None] * 20
file='c:\\aaa\\SQL_Server_Databases_Inventory.xlsx'

query = """SET NOCOUNT ON; EXEC DBA.dbo.Databases_Inventory;""".format(input)
print(query)

# Start writing to Excel
writer = pd.ExcelWriter(file, engine='xlsxwriter')

# Loop over all SQL Server Instances
for server in ('USSQL', 'BSQL', 'KSSQL', 'ZSSQL', 'WSSQL', 'SQLCLUST','TESTINGSQL', 'BIDEV16','DWH16-AZURE', 'BMANBAS', 'FSQL2', 'NEMALADB', 'BIGFIXMASTER'):
    # server parameters
    database = 'DBA' 
    username = 'naya5' 
    password = '9451P9Qt%&h!'
    conn_string =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    
    conn = pyodbc.connect(conn_string)
    conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string}) # To support pandas

    # --- Print using pandas dataframe ---
    print(f'Connecting to {server}')
    engine = sa.create_engine(conn_url)
    df = pd.read_sql_query(query, engine)
    appended_data[sheet_no] = df
    
    # Export DataFrame content
    sheet_name=server
    appended_data[sheet_no].to_excel(writer, sheet_name=sheet_name, startrow=1, header=False)
    
    # Get the xlsxwriter workbook and worksheet objects.
    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]

    # Add a header format.
    header_format = workbook.add_format({
            "valign": "vcenter",
            "align": "left",
            "bg_color": "#951F06",
            "bold": True,
            "font_color": "#FFFFFF",
            "border" : 1, 
            "border_color": "#D3D3D3"
            })

    row_format = workbook.add_format({
            'bg_color':'yellow'
            })

    # Write the column headers with the defined format.
    for col_num, value in enumerate(appended_data[sheet_no].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)

    # Close connection to database
    conn.close
    sheet_no = sheet_no + 1

writer.close()
    
# Open Excel to display
import os
os.startfile(file)


SET NOCOUNT ON; EXEC DBA.dbo.Databases_Inventory;
Connecting to USSQL
Connecting to BSQL
Connecting to KSSQL
Connecting to ZSSQL
Connecting to WSSQL
Connecting to SQLCLUST
Connecting to TESTINGSQL
Connecting to BIDEV16
Connecting to DWH16-AZURE
Connecting to BMANBAS
Connecting to FSQL2
Connecting to NEMALADB
Connecting to BIGFIXMASTER


Cretae the Stored Procedure that will query Inventory Data

In [36]:
# Read data from SQL Server
import pyodbc
import sqlalchemy as sa
from sqlalchemy.engine import URL       # To support pandas
from sqlalchemy import create_engine    # To support pandas

sqlCreateSP = "create procedure dbo.Databases_Inventory \
AS \
begin \
	if object_id('tempdb..#t', 'U') is not null \
	drop table #t; \
	create table #t( \
	  ServerName varchar(128) default @@servername \
	, Version varchar(2000) \
	, DBName varchar(128) default db_name() \
	, DBOwner varchar(128) \
	, CreateDate datetime2 \
	, RecoveryModel varchar(12) \
	, StateDesc varchar(60) \
	, CompatibilityLevel int \
	, DataFileSizeMB int \
	, LogFileSizeMB int \
	, DataUsageMB int \
	, IndexUsageMB int \
	, SizeMB decimal(17,2) \
	, Collation varchar(60) \
	, UserCount int \
	, RoleCount int \
	, TableCount int \
	, SPCount int \
	, UDFCount int \
	, ViewCount int \
	, DMLTriggerCount int \
	, IsCaseSensitive bit \
	, IsTrustWorthy bit \
	, LastFullBackupDate datetime2 \
	, LastDiffBackupDate datetime2 \
	, LastLogBackupDate datetime2); \
 \
	 \
	insert into #t(DBName, Version, DBOwner, CreateDate, RecoveryModel, StateDesc, CompatibilityLevel, IsCaseSensitive \
	, IsTrustWorthy, Collation, LastFullBackupDate, LastDiffBackupDate, LastLogBackupDate) \
	select name, @@version, suser_sname(owner_sid), create_date, recovery_model_desc, state_desc,compatibility_level \
	, IsCaseSensitive=CAST(CHARINDEX(N'_CS_', collation_name) AS bit), is_trustworthy_on, Collation_Name \
	, t.LastFullBackup, t.LastDiffBackup, t.LastLogBackup \
	from master.sys.databases db \
	outer apply( SELECT \
	MAX(CASE WHEN b.type = 'D' THEN b.backup_finish_date END) AS LastFullBackup, \
	MAX(CASE WHEN b.type = 'I' THEN b.backup_finish_date END) AS LastDiffBackup, \
	MAX(CASE WHEN b.type = 'L' THEN b.backup_finish_date END) AS LastLogBackup \
	FROM msdb.dbo.backupset b  \
	where b.database_name = db.name \
	) t; \
 \
	EXEC master.dbo.sp_msforeachdb'use [?] \
	update t set SizeMB=(select sum(size)/128. from dbo.sysfiles) \
	, DataUsageMB=x.DataUsageMB, IndexUsageMB=x.IndexUsageMB \
	   , DataFileSizeMB = u.DBSize, LogFileSizeMB = u.LogSize  \
	   , TableCount=y.TC, UDFCount=y.UC, SPCount = y.SC, ViewCount=y.VC \
	   , DMLTriggerCount=y.DC \
	   , UserCount = z.UC, RoleCount = z.RC \
	from #t t \
	   outer apply ( \
	   SELECT SUM(case when df.type in (0,2,4) then df.size else 0 end)/128 \
	   , SUM(case when df.type in (1,3) then df.size else 0 end)/128  \
	   FROM sys.database_files df  \
	   ) u(DBSize, LogSize) \
	   outer apply(select  DataUsageMB=sum( \
		CASE \
		When it.internal_type IN (202,204,207,211,212,213,214,215,216,221,222,236) Then 0 \
		When a.type <> 1 and p.index_id < 2 Then a.used_pages \
		When p.index_id < 2 Then a.data_pages \
		Else 0 \
		END)/128, \
	IndexUsageMB=(sum(a.used_pages)-sum( \
		CASE \
		When it.internal_type IN (202,204,207,211,212,213,214,215,216,221,222,236) Then 0 \
		When a.type <> 1 and p.index_id < 2 Then a.used_pages \
		When p.index_id < 2 Then a.data_pages \
		Else 0 \
		END \
		))/128 \
		from sys.partitions p join sys.allocation_units a on p.partition_id = a.container_id \
		left join sys.internal_tables it on p.object_id = it.object_id \
	   ) x \
	   outer apply  \
	   ( select SC=Sum(case Type when ''P'' then 1 else 0 end) \
		, DC=Sum(case Type when ''TR'' then 1 else 0 end) \
		, TC=Sum(case Type when ''U'' then 1 end) \
		, UC= sum(case when Type in (''TF'', ''IF'', ''FN'') then 1 else 0 end) \
		, VC=Sum(case Type when ''V'' then 1 else 0 end) \
		from sys.objects where object_id > 1024 \
		and type in (''U'',''P'',''TR'',''V'',''TF'',''IF'',''FN'') \
	   ) y \
	   outer apply  \
	   ( select UC = sum(case when [Type] in (''G'',''S'',''U'') then 1 else 0 end) \
		  , RC = sum(case when Type = ''R'' then 1 else 0 end) \
		  from sys.database_principals \
		  where principal_id > 4 \
	   ) z where t.DBName=db_name(); \
	' \
SELECT * FROM #t; \
END"

# print(sqlCreateSP)

# Stored Procedure Drop Statement
sqlDropSP=(f"IF EXISTS (SELECT * FROM sys.objects             \
            WHERE type='P' AND name='Databases_Inventory')   \
           DROP PROCEDURE dbo.Databases_Inventory")

# Stored Procedure Call Statement
sqlExecSP="{call Databases_Inventory}"

# Loop over all SQL Server Instances
for server in ('USSQL', 'BSQL', 'KSSQL', 'ZSSQL', 'WSSQL', 'SQLCLUST','BIDEV16','DWH16-AZURE'):
    # server parameters
    database = 'DBA' 
    username = 'naya5' 
    password = '9451P9Qt%&h!'
    conn_string =  'DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password
    
    conn = pyodbc.connect(conn_string, autocommit=True)
    conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_string}) # To support pandas

    cursor=conn.cursor()
    print("\nCreating Stored Procedure Databases_Inventory on Server " + server)
    
    # Drop SP if exists
    cursor.execute(sqlDropSP)
    
    # Create SP using Create statement
    cursor.execute(sqlCreateSP)
    
    # Call SP and trap Error if raised
    cursor.execute(sqlExecSP)





Creating Stored Procedure Databases_Inventory on Server USSQL

Creating Stored Procedure Databases_Inventory on Server BSQL

Creating Stored Procedure Databases_Inventory on Server KSSQL

Creating Stored Procedure Databases_Inventory on Server ZSSQL

Creating Stored Procedure Databases_Inventory on Server WSSQL

Creating Stored Procedure Databases_Inventory on Server SQLCLUST

Creating Stored Procedure Databases_Inventory on Server BIDEV16

Creating Stored Procedure Databases_Inventory on Server DWH16-AZURE
